In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [3]:
!tar -xf spark-3.3.2-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [9]:
spark

In [10]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xf ngrok-v3-stable-linux-amd64.tgz

--2023-03-12 17:31:29--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9300212 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.87M  43.3MB/s    in 0.2s    

2023-03-12 17:31:30 (43.3 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9300212/9300212]



In [11]:
# This is my token, you can setup your own ngrok account and get your own if you want
# I'm not sure if I can launch tunnels for everyone in class, no biggie at this point if you can't get the UI up.
# https://ngrok.com
!./ngrok config add-authtoken 2MeuAq8RhqK5mcT1MlX8rrDLLUq_48rPK2gwQGdFeQQWPcrTL

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
# start ngrok, be sure to verify ngrok email they sent you if you signed up.
get_ipython().system_raw('./ngrok http 4050 &')

In [13]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [14]:
!pip install awscli
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 38.7 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 3.4 MB/s eta 0:00:00


In [15]:
!aws configure

AWS Access Key ID [None]: EJO67TBP7ZHN4KJIOMJU
AWS Secret Access Key [None]: lqZUzZIK2VysIX4eRkXiCrH7w0HiOs5JeWF1QFUH
Default region name [None]: us-east-1
Default output format [None]: None


In [16]:
import boto3
s3 = boto3.client('s3',endpoint_url='https://s3.wasabisys.com')
response = s3.list_buckets()

In [17]:
response['Buckets']

[{'Name': 'data603',
  'CreationDate': datetime.datetime(2023, 2, 5, 19, 38, 12, tzinfo=tzlocal())},
 {'Name': 'data603-spring2023',
  'CreationDate': datetime.datetime(2023, 2, 5, 19, 6, 49, tzinfo=tzlocal())}]

In [18]:
objects = s3.list_objects_v2(Bucket='data603-spring2023')

for obj in objects['Contents']:
  print(obj['Key'])

assignment3/
assignment3/test-annotations-bbox.csv
assignment3/train-annotations-bbox.csv
assignment3/validation-annotations-bbox.csv
classexercise1/
classexercise1/bible.txt
classexercise1/mapreduce.py
homework1/
homework1/nfl_salary.csv


In [19]:
s3r = boto3.resource('s3', endpoint_url='https://s3.wasabisys.com')

for bucket in s3r.buckets.all():
  print(bucket.name)

data603
data603-spring2023


In [20]:
bucket = s3r.Bucket('data603-spring2023')

for obj in bucket.objects.all():
  print(obj.key)

assignment3/
assignment3/test-annotations-bbox.csv
assignment3/train-annotations-bbox.csv
assignment3/validation-annotations-bbox.csv
classexercise1/
classexercise1/bible.txt
classexercise1/mapreduce.py
homework1/
homework1/nfl_salary.csv


In [21]:
s3r.Bucket('data603-spring2023').download_file("assignment3/test-annotations-bbox.csv", "/content/drive/MyDrive/Colab Notebooks/603 Big data/Data Files/test-annotations-bbox.csv")
s3r.Bucket('data603-spring2023').download_file("assignment3/train-annotations-bbox.csv", "/content/drive/MyDrive/Colab Notebooks/603 Big data/Data Files/train-annotations-bbox.csv")
s3r.Bucket('data603-spring2023').download_file("assignment3/validation-annotations-bbox.csv", "/content/drive/MyDrive/Colab Notebooks/603 Big data/Data Files/validation-annotations-bbox.csv")


In [22]:
# bounding_boxes = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/603 Big data/Data Files/test-annotations-bbox.csv', header = True)
bb_test = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/603 Big data/Data Files/test-annotations-bbox.csv', header = True)
bb_train = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/603 Big data/Data Files/train-annotations-bbox.csv', header = True)
bb_validation = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/603 Big data/Data Files/validation-annotations-bbox.csv', header = True)

In [23]:
bounding_boxes = bb_test.union(bb_train).union(bb_validation)

In [24]:
bounding_boxes.show(10)

+----------------+------+---------+----------+-----------+----------+----------+-----------+----------+-----------+---------+-----------+--------+
|         ImageID|Source|LabelName|Confidence|       XMin|      XMax|      YMin|       YMax|IsOccluded|IsTruncated|IsGroupOf|IsDepiction|IsInside|
+----------------+------+---------+----------+-----------+----------+----------+-----------+----------+-----------+---------+-----------+--------+
|000026e7ee790996|xclick| /m/07j7r|         1|   0.071875| 0.1453125|   0.20625| 0.39166668|         0|          1|        1|          0|       0|
|000026e7ee790996|xclick| /m/07j7r|         1|  0.4390625|  0.571875|0.26458332| 0.43541667|         0|          1|        1|          0|       0|
|000026e7ee790996|xclick| /m/07j7r|         1|    0.66875|         1|         0|  0.5520833|         0|          1|        1|          0|       0|
|000062a39995e348|xclick| /m/015p6|         1| 0.20620842|  0.849224|0.15463917|          1|         0|          0|   

In [25]:
bb_test.count() + bb_validation.count() + bb_train.count()

15851536

In [26]:
bounding_boxes.count()

15851536

In [27]:
bounding_boxes.limit(20).toPandas()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000026e7ee790996,xclick,/m/07j7r,1,0.071875,0.1453125,0.20625,0.39166668,0,1,1,0,0
1,000026e7ee790996,xclick,/m/07j7r,1,0.4390625,0.571875,0.26458332,0.43541667,0,1,1,0,0
2,000026e7ee790996,xclick,/m/07j7r,1,0.66875,1,0,0.5520833,0,1,1,0,0
3,000062a39995e348,xclick,/m/015p6,1,0.20620842,0.849224,0.15463917,1,0,0,0,0,0
4,000062a39995e348,xclick,/m/05s2s,1,0.13747229,0.37694013,0,0.88365245,1,1,0,0,0
5,0000c64e1253d68f,xclick,/m/0k4j,1,0,0.5132743,0.32079646,0.6902655,0,1,0,0,0
6,0000c64e1253d68f,xclick,/m/0k4j,1,0.016224189,0.26843658,0.29867256,0.46238938,1,0,0,0,0
7,0000c64e1253d68f,xclick,/m/0k4j,1,0.48082596,0.9041298,0.23230088,0.48893806,1,0,0,0,0
8,0000c64e1253d68f,xclick,/m/0k4j,1,0.7522124,1,0.3318584,0.65707964,0,1,0,0,0
9,0000c64e1253d68f,xclick,/m/07yv9,1,0,0.9734513,0,0.044247787,0,1,1,0,0


In [28]:
# bounding_boxes.sort(bounding_boxes.XMin.desc()).collect()
# bounding_boxes.sort("XMin", ascending=False).collect()

In [29]:
bounding_boxes.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- LabelName: string (nullable = true)
 |-- Confidence: string (nullable = true)
 |-- XMin: string (nullable = true)
 |-- XMax: string (nullable = true)
 |-- YMin: string (nullable = true)
 |-- YMax: string (nullable = true)
 |-- IsOccluded: string (nullable = true)
 |-- IsTruncated: string (nullable = true)
 |-- IsGroupOf: string (nullable = true)
 |-- IsDepiction: string (nullable = true)
 |-- IsInside: string (nullable = true)



In [30]:
type(bounding_boxes)

pyspark.sql.dataframe.DataFrame

In [31]:
from pyspark.sql.types import DoubleType

bounding_boxes = bounding_boxes.withColumn("XMin", bounding_boxes['XMin'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("XMax", bounding_boxes['XMax'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("YMin", bounding_boxes['YMin'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("YMax", bounding_boxes['YMax'].cast(DoubleType()))
bounding_boxes = bounding_boxes.withColumn("Confidence", bounding_boxes['Confidence'].cast(DoubleType()))

In [32]:
bounding_boxes.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- LabelName: string (nullable = true)
 |-- Confidence: double (nullable = true)
 |-- XMin: double (nullable = true)
 |-- XMax: double (nullable = true)
 |-- YMin: double (nullable = true)
 |-- YMax: double (nullable = true)
 |-- IsOccluded: string (nullable = true)
 |-- IsTruncated: string (nullable = true)
 |-- IsGroupOf: string (nullable = true)
 |-- IsDepiction: string (nullable = true)
 |-- IsInside: string (nullable = true)



In [33]:
sorteddf = bounding_boxes.sort("XMin", ascending=False).show()

+----------------+------+----------+----------+--------+----+--------+--------+----------+-----------+---------+-----------+--------+
|         ImageID|Source| LabelName|Confidence|    XMin|XMax|    YMin|    YMax|IsOccluded|IsTruncated|IsGroupOf|IsDepiction|IsInside|
+----------------+------+----------+----------+--------+----+--------+--------+----------+-----------+---------+-----------+--------+
|df28cb59b47aa0c0|xclick|  /m/07j7r|       1.0|0.999375| 1.0|     0.0|0.255159|         0|          1|        0|          0|       0|
|7b0eafddaeb725cc|xclick|/m/04dr76w|       1.0|0.999375| 1.0|  0.2625|  0.2825|         0|          0|        0|          0|       0|
|a49c40de46f0f01a|xclick|  /m/05s2s|       1.0|0.999375| 1.0|0.670759|0.698661|         0|          0|        0|          0|       0|
|9618684128b830c7|xclick|  /m/083wq|       1.0|0.999375| 1.0|0.247191|0.261236|         1|          0|        0|          0|       0|
|fab15cce08eec8e4|xclick|  /m/07j7r|       1.0|0.999375| 1.0|0

In [34]:
sorteddf = bounding_boxes.sort("XMin", ascending=False)

In [35]:
sorteddf.show(5)

+----------------+------+---------+----------+--------+----+--------+--------+----------+-----------+---------+-----------+--------+
|         ImageID|Source|LabelName|Confidence|    XMin|XMax|    YMin|    YMax|IsOccluded|IsTruncated|IsGroupOf|IsDepiction|IsInside|
+----------------+------+---------+----------+--------+----+--------+--------+----------+-----------+---------+-----------+--------+
|50241898b7164012|xclick| /m/07j7r|       1.0|0.999375| 1.0|0.510748|0.760963|         1|          0|        0|          0|       0|
|4025fc558f8ba0b0|xclick|/m/08dz3q|       1.0|0.999375| 1.0|0.469541|0.999063|         0|          0|        0|          0|       0|
|398f175cb4f61543|xclick| /m/09j5n|       1.0|0.999375| 1.0|0.641917|0.662594|         0|          1|        0|          0|       0|
|4f5621cc67c13ce6|xclick| /m/09j2d|       1.0|0.999375| 1.0|0.500942|0.702448|         1|          0|        0|          0|       0|
|0901d293ba2c9106|xclick| /m/0d4v4|       1.0|0.999375| 1.0|0.467521|

In [36]:
pdf = sorteddf.limit(20).toPandas()
type(pdf)

pandas.core.frame.DataFrame

In [37]:
pdf

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,fab15cce08eec8e4,xclick,/m/07j7r,1.0,0.999375,1.0,0.549834,0.999169,1,0,0,0,0
1,9618684128b830c7,xclick,/m/083wq,1.0,0.999375,1.0,0.247191,0.261236,1,0,0,0,0
2,cf06075f20f2df36,xclick,/m/0dzf4,1.0,0.999375,1.0,0.367778,0.437778,1,0,0,0,0
3,74eac9b39ab1d708,xclick,/m/05s2s,1.0,0.999375,1.0,0.589167,0.603333,1,1,0,0,0
4,df28cb59b47aa0c0,xclick,/m/07j7r,1.0,0.999375,1.0,0.000000,0.255159,0,1,0,0,0
5,a49c40de46f0f01a,xclick,/m/05s2s,1.0,0.999375,1.0,0.670759,0.698661,0,0,0,0,0
6,fa07d530f52bae24,xclick,/m/079cl,1.0,0.999375,1.0,0.345000,0.740000,0,0,0,0,0
7,398f175cb4f61543,xclick,/m/09j5n,1.0,0.999375,1.0,0.641917,0.662594,0,1,0,0,0
8,50241898b7164012,xclick,/m/07j7r,1.0,0.999375,1.0,0.510748,0.760963,1,0,0,0,0
9,4f5621cc67c13ce6,xclick,/m/09j2d,1.0,0.999375,1.0,0.500942,0.702448,1,0,0,0,0
